In [1]:
import requests
import pandas as pd
import numpy as np
import itertools
import copy

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# FPL API URL
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# JSON keys
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
# storing json outputs as dataframes

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [4]:
# Pulling in player position into slim_elements_df

elements_df['position'] = elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
elements_df['team_name'] = elements_df.team.map(teams_df.set_index('id').name)

In [5]:
# Filtering out only the necessary columns

slim_elements_df = elements_df[['first_name', 'second_name','team_name','position','news','selected_by_percent','in_dreamteam',
                                'now_cost','form','points_per_game','minutes','goals_scored','assists','clean_sheets',
                                'goals_conceded','clean_sheets','goals_conceded','yellow_cards','red_cards','saves','bonus',
                                'transfers_in','value_season','total_points','influence','creativity','threat','ict_index']]


In [6]:
# convering columns into numeric data type

slim_elements_df['selected_by_percent'] = pd.to_numeric(slim_elements_df['selected_by_percent'])
slim_elements_df['form'] = pd.to_numeric(slim_elements_df['form'])
slim_elements_df['points_per_game'] = pd.to_numeric(slim_elements_df['points_per_game'])
slim_elements_df['value_season'] = pd.to_numeric(slim_elements_df['value_season'])
slim_elements_df['influence'] = pd.to_numeric(slim_elements_df['influence'])
slim_elements_df['creativity'] = pd.to_numeric(slim_elements_df['creativity'])
slim_elements_df['threat'] = pd.to_numeric(slim_elements_df['threat'])
slim_elements_df['ict_index'] = pd.to_numeric(slim_elements_df['ict_index'])

In [7]:
# actual cost of the player is now_cost/10
slim_elements_df['actual_cost'] = slim_elements_df['now_cost']/10

# creating additional metrics
slim_elements_df['games_completed'] = slim_elements_df['minutes']/90
slim_elements_df['points_per_90_mins'] = slim_elements_df['total_points']/slim_elements_df['games_completed']
slim_elements_df['ga_per_90_mins'] = (slim_elements_df['goals_scored']+slim_elements_df['assists'])/slim_elements_df['games_completed']
slim_elements_df['points_per_million'] = slim_elements_df['total_points']/slim_elements_df['actual_cost']

# eligible players
eligible_players = slim_elements_df[slim_elements_df['news'] == '']

In [8]:
differentials = slim_elements_df.loc[(slim_elements_df['news'] == '') & (slim_elements_df['selected_by_percent'] <= 20) & (slim_elements_df['games_completed'] >= 20)]


In [9]:
def knapsack_solution(players, player_costs, player_values, max_cost, count):
    
    """
    function that returns the knapsack cost matrix
    """
  
    num_players = len(players)
  
    cost_matrix = [[[0 for k in range(count+1)] for j in range(max_cost+1)] for i in range(num_players)]
    
    for i in range(num_players):
        for j in range(max_cost+1):
            for k in range(count+1):
                if (player_costs[i] > j) or (1 > k):
                    cost_matrix[i][j][k] = cost_matrix[i-1][j][k]
                else: 
                    cost_matrix[i][j][k] = max(cost_matrix[i-1][j][k], player_values[i]+cost_matrix[i-1][j-player_costs[i]][k-1])

    return cost_matrix
    

In [10]:
def get_used_items(players, player_costs, player_values, max_cost, count, cost_matrix):
    
    """
    function that returns the used players from the cost matrix
    """
    
    playerIndex = len(players) - 1
    
    currentCost = -1
    currentCount = count
    marked = [0 for k in range(len(players))]

    bestValue = -1
    
    for j in range(max_cost+1):
        value = cost_matrix[playerIndex][j][count]
        if (bestValue == -1) or (value > bestValue):
            currentCost = j
            bestValue = value
    
    while (playerIndex >= 0 and currentCost >= 0 and currentCount >= 0):
        if (playerIndex == 0 and cost_matrix[playerIndex][currentCost][currentCount] > 0) or (cost_matrix[playerIndex][currentCost][currentCount] != cost_matrix[playerIndex-1][currentCost][currentCount]):
            marked[playerIndex] = 1
            currentCost = currentCost - player_costs[playerIndex]
            currentCount = currentCount - 1
        playerIndex = playerIndex - 1

    return marked
      

In [11]:
def optimum_keeping(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    gk_df = eligible_players[eligible_players['position'] == 'Goalkeeper']
    gk_df = gk_df.reset_index()
    goalkeepers = gk_df.index.tolist()
    goalkeeper_costs = (gk_df['now_cost']).tolist()
    goalkeeper_values = gk_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(goalkeepers, goalkeeper_costs, goalkeeper_values, max_cost, 2)
    
    used_players = get_used_items(goalkeepers, goalkeeper_costs, goalkeeper_values, max_cost, 2, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, gk_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [12]:
def optimum_defence(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    def_df = eligible_players[eligible_players['position'] == 'Defender']
    def_df = def_df.reset_index()
    defenders = def_df.index.tolist()
    defender_costs = (def_df['now_cost']).tolist()
    defender_values = def_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(defenders, defender_costs, defender_values, max_cost, 5)
    
    used_players = get_used_items(defenders, defender_costs, defender_values, max_cost, 5, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, def_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [13]:
def optimum_midfield(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    mid_df = eligible_players[eligible_players['position'] == 'Midfielder']
    mid_df = mid_df.reset_index()
    midfielders = mid_df.index.tolist()
    midfielder_costs = (mid_df['now_cost']).tolist()
    midfielder_values = mid_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(midfielders, midfielder_costs, midfielder_values, max_cost, 5)
    
    used_players = get_used_items(midfielders, midfielder_costs, midfielder_values, max_cost, 5, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, mid_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [14]:
def optimum_attack(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    att_df = eligible_players[eligible_players['position'] == 'Forward']
    att_df = att_df.reset_index()
    attackers = att_df.index.tolist()
    attacker_costs = (att_df['now_cost']).tolist()
    attacker_values = att_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(attackers, attacker_costs, attacker_values, max_cost, 3)
    
    used_players = get_used_items(attackers, attacker_costs, attacker_values, max_cost, 3, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, att_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [15]:
def print_all_sum_rec(target, current_sum, start, output, result):
    if current_sum == target:
        output.append(copy.copy(result))

    for i in range(start, target):
        temp_sum = current_sum + i
        if temp_sum <= target:
            result.append(i)
            print_all_sum_rec(target, temp_sum, i, output, result)
            result.pop()
        else:
            return

def print_all_sum(target):
    output = []
    result = []
    print_all_sum_rec(target, 0, 4, output, result)
    return output


In [16]:
def cost_breakdown(number):
    breakdown = print_all_sum(number)
    combinations = []
    for i in breakdown:
        if len(i) == 4:
            if i[0] >= 8 and i[1] >= 25 and i[2] >= 30 and i[3] >= 20:
                combinations.append(i)
    return combinations

In [17]:
def optimize(opt_metric):
    costs_combinations = cost_breakdown(100)

    comb_df = pd.DataFrame(columns = ['costs', 'total_cost', opt_metric])
    
    for costs in costs_combinations:
        
        gk = optimum_keeping(eligible_players, costs[0], opt_metric)
        dfnc = optimum_defence(eligible_players, costs[1], opt_metric)
        mid = optimum_midfield(eligible_players, costs[2], opt_metric)
        att = optimum_attack(eligible_players, costs[3], opt_metric)
        
        final = pd.concat([gk, dfnc, mid, att])
        total_cost = final['actual_cost'].sum()
        optimised_metric = final[opt_metric].sum()
        cost_details = [costs, total_cost, optimised_metric]
        
        comb_df.loc[len(comb_df)] = cost_details

    return comb_df

In [18]:
combs = optimize('points_per_million')
combs['points_per_million'] = pd.to_numeric(combs['points_per_million'])
combs.sort_values(by=['points_per_million'], ascending=False).head(20)

In [ ]:
optimum_keeping(eligible_players, 10, 'points_per_million')
optimum_defence(eligible_players, 25, 'points_per_million')
optimum_midfield(eligible_players, 35, 'points_per_million')
optimum_attack(eligible_players, 35, 'points_per_million')